In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import mlflow
import plotly.graph_objects as go
import plotly.express as px
import mlflow.sklearn

from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_parquet('dataset_full.parquet')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', '{:.4f}'.format)

print(f"Structure du dataset : {df.shape[0]} lignes et {df.shape[1]} colonnes")
display(df.head())

In [ ]:
df.dtypes

---

##### Drop des colonnes inutiles au modèle

In [ ]:
# 1. NETTOYAGE GÉOGRAPHIQUE & RISQUE DÉPARTEMENTAL

df['Code du département de l\'établissement'] = df['Code du département de l\'établissement'].astype(str).str.zfill(2)

dep_risk_map = df.groupby("Code du département de l'établissement")["fermeture"].mean()
df['risque_departemental'] = df['Code du département de l\'établissement'].map(dep_risk_map)

# 2. TRAITEMENT DES TYPES (Catégories et Nullables)

df['Catégorie juridique de l\'unité légale'] = df['Catégorie juridique de l\'unité légale'].astype(str)
df['age_estime'] = df['age_estime'].astype(float)
df['Tranche_effectif_num'] = df['Tranche_effectif_num'].fillna(0).astype(float)

# 3. ENCODAGE DES VARIABLES

df['is_ess'] = df['Economie sociale et solidaire unité légale'].map({'O': 1, 'N': 0}).fillna(0)

df_final = pd.get_dummies(
    df, 
    columns=['libelle_section_ape', 'Catégorie juridique de l\'unité légale'], 
    prefix=['APE', 'CJ'],
    drop_first=True
)

# 4. SÉLECTION FINALE ET NETTOYAGE DES COLONNES INUTILES
cols_to_drop = [
    'SIREN', 'Code postal de l\'établissement', 'Code commune de l\'établissement',
    'Dénomination de l\'unité légale', 'Activité principale de l\'unité légale',
    'Date_fermeture_finale', 'latitude', 'longitude', 'code_ape',
    'Code du département de l\'établissement', 'Code de la région de l\'établissement',
    'Economie sociale et solidaire unité légale'
]

df_final = df_final.drop(columns=[c for c in cols_to_drop if c in df_final.columns])

# 5. DERNIERS RÉGLAGES POUR LA SURVIE

df_final['fermeture'] = df_final['fermeture'].astype(bool)
df_final = df_final[df_final['age_estime'] > 0]

print(f"✅ Dataset finalisé : {df_final.shape[0]} lignes, {df_final.shape[1]} colonnes.")
display(df_final.head(10))

---

In [ ]:
# 1. Calcul de la fréquence pour les colonnes binaires (APE et CJ)
binary_cols = [c for c in df_final.columns if c.startswith('APE_') or c.startswith('CJ_')]
frequencies = df_final[binary_cols].mean().sort_values(ascending=False) * 100

# 2. Visualisation des 30 catégories les plus rares vs les plus fréquentes

# 3. Focus sur les "Micro-Catégories"
rare_limit = 0.1 # Seuil de 0.1%
rare_cols = frequencies[frequencies < rare_limit]

print(f"--- 🔍 Analyse des colonnes rares (< {rare_limit}%) ---")
print(f"Il y a {len(rare_cols)} colonnes qui concernent moins de 0.1% du dataset.")
if len(rare_cols) > 0:
    print(rare_cols)

In [ ]:
# 1. Identifier les colonnes à fusionner par famille
rare_ape_cols = [c for c in rare_cols.index if c.startswith('APE_')]
rare_cj_cols = [c for c in rare_cols.index if c.startswith('CJ_')]

# 2. Créer la colonne "Autres" pour les APE
if rare_ape_cols:
    df_final['APE_Autres_Secteurs'] = df_final[rare_ape_cols].any(axis=1)
    df_final.drop(columns=rare_ape_cols, inplace=True)

# 3. Créer la colonne "Autres" pour les CJ
if rare_cj_cols:
    df_final['CJ_Autres_Status'] = df_final[rare_cj_cols].any(axis=1)
    df_final.drop(columns=rare_cj_cols, inplace=True)

print(f"✅ Nettoyage terminé. Nouveau nombre de colonnes : {len(df_final.columns)}")

In [ ]:
# 1. Calcul des fréquences en nombre d'entreprises
binary_cols = [c for c in df_final.columns if c.startswith('APE_')]
counts = df_final[binary_cols].sum().sort_values(ascending=True)

niche_sectors = counts[counts < 6000] 

fig_niche = px.bar(
    x=niche_sectors.values, 
    y=niche_sectors.index,
    orientation='h',
    title=f"🔍 Zoom sur les {len(niche_sectors)} secteurs de niche (< 0.5% du total)",
    labels={'x': 'Nombre total d\'établissements', 'y': 'Secteur APE'},
    color=niche_sectors.values,
    color_continuous_scale='Reds_r'
)

fig_niche.update_layout(
    height=800, 
    margin=dict(l=400),
    xaxis_title="Nombre d'entreprises (Volume réel)"
)

fig_niche.show()

In [ ]:
df_final.head()

---

##### Visu des facteurs d'influence

In [ ]:
corr_target = df_final.corr()['fermeture'].sort_values(ascending=False).to_frame()

fig_corr = go.Figure(data=go.Heatmap(
    z=corr_target.values,
    x=['Corrélation'],
    y=corr_target.index,
    colorscale='RdBu_r',
    zmin=-0.15, zmax=0.15,
    text=corr_target.values.round(3),
    texttemplate="%{text}",
    showscale=True
))

fig_corr.update_layout(
    title="🎯 Focus : Impact des variables sur la Fermeture",
    height=1400,
    width=800,
    margin=dict(l=450, r=50, t=100, b=100),
    yaxis=dict(
        side='left',
        tickfont=dict(size=12),
        autorange="reversed"
    )
)

fig_corr.show()

---

In [ ]:
# 1. Préparation des matrices (X = features, y = structuré pour la survie)
X = df_final.drop(columns=['fermeture', 'age_estime'])

# sksurv a besoin d'un format spécifique : un tableau de tuples (Event, Time)
y = np.array([(bool(f), float(a)) for f, a in zip(df_final['fermeture'], df_final['age_estime'])],
             dtype=[('event', 'bool'), ('time', 'float')])

# 2. Split 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Lancement du Run MLflow
mlflow.set_experiment("Survie_Entreprises_V1")

with mlflow.start_run(run_name="Baseline_RSF_57_cols"):
    # On commence "léger" pour ne pas faire chauffer la machine
    rf_params = {
        "n_estimators": 50,
        "max_depth": 8,
        "min_samples_split": 100,
        "n_jobs": -1
    }
    mlflow.log_params(rf_params)
    
    rsf = RandomSurvivalForest(**rf_params)
    rsf.fit(X_train, y_train)
    
    # Score de concordance (C-index)
    c_index = rsf.score(X_test, y_test)
    mlflow.log_metric("c_index", c_index)
    
    # Sauvegarde
    mlflow.sklearn.log_model(rsf, "model_rsf")
    
    print(f"✅ Entraînement terminé ! C-Index: {c_index:.3f}")

In [ ]:
# 1. Définir les points temporels (en mois)
times = [12, 24, 36]

# 2. Prédire les fonctions de survie sur le jeu de test

surv_funcs = rsf.predict_survival_function(X_test)

# 3. Extraire les probabilités de FERMETURE (1 - Survie)

risques_data = []
for fn in surv_funcs:
    risques_data.append({
        "Risque_1_an": round((1 - fn(12)) * 100, 2),
        "Risque_2_ans": round((1 - fn(24)) * 100, 2),
        "Risque_3_ans": round((1 - fn(36)) * 100, 2)
    })

df_predictions = pd.DataFrame(risques_data, index=X_test.index)

# 4. Afficher un aperçu
print("📋 Aperçu des probabilités de fermeture (%) :")
display(df_predictions.head(10))

# 5. Petite stat rapide pour voir si c'est cohérent
print("\n📊 Risque moyen par horizon :")
print(df_predictions.mean())

In [ ]:
# 1. On fusionne les prédictions avec les caractéristiques d'origine
df_analyse = df_predictions.merge(X_test, left_index=True, right_index=True)

# 2. On compare les moyennes des deux groupes
high_risk = df_analyse[df_analyse['Risque_3_ans'] > 90].mean()
low_risk = df_analyse[df_analyse['Risque_3_ans'] < 75].mean()

comparison = pd.DataFrame({'Ultra-Risque (>90%)': high_risk, 'Plus Solides (<75%)': low_risk})
print("🔍 Comparaison des profils types :")
display(comparison.head(15))

In [ ]:
# 1. On récupère les lignes de test avec le risque_departemental d'origine
df_geo_check = df_analyse[['Risque_3_ans', 'risque_departemental']].copy()

# 2. On crée des groupes de risque pour y voir plus clair
df_geo_check['Tranche_Risque'] = pd.qcut(df_geo_check['Risque_3_ans'], q=5, 
                                         labels=['Très Faible', 'Faible', 'Moyen', 'Fort', 'Critique'])

# 3. On regarde la valeur moyenne du score départemental pour chaque tranche
geo_stats = df_geo_check.groupby('Tranche_Risque')['risque_departemental'].mean()

print("📍 Score de risque départemental moyen par tranche de prédiction :")
print(geo_stats)

# 4. Visualisation
plt.figure(figsize=(10, 6))
sns.barplot(x=geo_stats.index, y=geo_stats.values, palette='OrRd')
plt.title('Validation : Le poids de la géographie dans la prédiction finale')
plt.ylabel('Moyenne du Score Risque Départemental (Target Encoded)')
plt.show()

In [ ]:
# On trie pour avoir les extrêmes
top_solides = df_analyse.sort_values('Risque_3_ans', ascending=True).head(10)
top_fragiles = df_analyse.sort_values('Risque_3_ans', ascending=False).head(10)

print("💎 TOP 10 : Les Entreprises les plus robustes")
display(top_solides[['Risque_1_an', 'Risque_3_ans', 'Tranche_effectif_num', 'risque_departemental']])

print("\n🔥 TOP 10 : Les Entreprises les plus fragiles")
display(top_fragiles[['Risque_1_an', 'Risque_3_ans', 'Tranche_effectif_num', 'risque_departemental']])

---